In [7]:
import streamlit as st
import yfinance as yf
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from datetime import date

# -------------------------------
# Page Settings & Custom CSS
# -------------------------------
st.set_page_config(page_title="Market Sentiment & Stock Prediction App", layout="wide")
st.markdown(
    """
    <style>
    .header {
        background-color: #4CAF50;
        padding: 15px;
        border-radius: 10px;
        color: white;
        text-align: center;
        margin-bottom: 20px;
    }
    .recommendation {
        background-color: #d4edda;
        padding: 10px;
        border-radius: 8px;
        border-left: 5px solid #28a745;
    }
    </style>
    """,
    unsafe_allow_html=True
)
st.markdown('<div class="header"><h1>Market Sentiment & Stock Prediction App</h1></div>', unsafe_allow_html=True)

# -------------------------------
# FinBERT Setup & Functions
# -------------------------------
@st.cache_resource(show_spinner=False)
def load_finbert_model():
    model_name = "yiyanghkust/finbert-tone"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return tokenizer, model, device

finbert_tokenizer, finbert_model, finbert_device = load_finbert_model()

def predict_sentiment(text: str):
    """
    Use FinBERT to predict sentiment on the given text.
    Returns a dict with sentiment label, confidence, and probabilities.
    """
    if not text.strip():
        return {"sentiment": "No News", "confidence": 0.0, "probabilities": {}}
    inputs = finbert_tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )
    inputs = {k: v.to(finbert_device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = finbert_model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    probs = probabilities.cpu().numpy()[0]
    pred_class = probs.argmax()
    label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return {
        "sentiment": label_mapping[pred_class],
        "confidence": float(probs[pred_class]),
        "probabilities": {label_mapping[i]: float(probs[i]) for i in range(len(probs))}
    }

# -------------------------------
# Data Fetching Functions
# -------------------------------
def get_company_data(ticker_symbol: str):
    """
    Fetch real-time news and basic price info using yfinance.
    Returns a dict with news headlines/text and price info.
    """
    ticker = yf.Ticker(ticker_symbol)
    news_data = ticker.news
    if news_data:
        headlines = [article.get("title", "") for article in news_data][:10]
        news_text = "\n".join(headlines)
    else:
        headlines = ["No recent news found."]
        news_text = ""
    hist_recent = ticker.history(period="5d")
    if not hist_recent.empty:
        latest = hist_recent.iloc[-1]
        price_info = {
            "Current Price": latest["Close"],
            "High": latest["High"],
            "Low": latest["Low"],
            "Volume": int(latest["Volume"])
        }
    else:
        price_info = {}
    return {
        "ticker": ticker_symbol,
        "news_headlines": headlines,
        "news_text": news_text,
        "price_info": price_info
    }

def get_historical_data(ticker_symbol: str, start: date, end: date):
    """
    Retrieve historical closing prices over a custom date range.
    """
    ticker = yf.Ticker(ticker_symbol)
    hist = ticker.history(start=start, end=end)
    if not hist.empty:
        return hist[["Close"]]
    return pd.DataFrame()

# -------------------------------
# Momentum Calculation Function
# -------------------------------
def compute_momentum(hist_df):
    """
    Compute momentum by fitting a linear regression on historical closing prices.
    Returns the slope of the trend line.
    """
    if hist_df.empty:
        return 0.0
    dates = np.array([d.toordinal() for d in hist_df.index]).reshape(-1, 1)
    prices = hist_df.iloc[:, 0].values.reshape(-1, 1)
    lr = LinearRegression()
    lr.fit(dates, prices)
    return lr.coef_[0][0]

# Mapping of sentiment to numeric score.
sentiment_mapping = {"Negative": -1, "Neutral": 0, "Positive": 1}

# -------------------------------
# Company Name Mapping
# -------------------------------
COMPANY_TO_TICKER = {
    "APPLE": "AAPL",
    "APPLE INC": "AAPL",
    "MICROSOFT": "MSFT",
    "MICROSOFT CORPORATION": "MSFT",
    "AMAZON": "AMZN",
    "AMAZON.COM, INC.": "AMZN",
    "ALPHABET": "GOOGL",
    "ALPHABET INC": "GOOGL",
    "TESLA": "TSLA",
    "TESLA, INC.": "TSLA",
}

def company_name_to_ticker(company_name: str):
    return COMPANY_TO_TICKER.get(company_name.upper())

# -------------------------------
# Sidebar Widgets for Enhanced Interactivity
# -------------------------------
st.sidebar.header("Customize Your Analysis")
company_options = list(COMPANY_TO_TICKER.keys())
selected_companies = st.sidebar.multiselect("Select Companies", options=company_options, default=["APPLE", "TESLA"])
st.sidebar.markdown("### Historical Data Range")
start_date = st.sidebar.date_input("Start Date", value=pd.to_datetime("2020-01-01"))
end_date = st.sidebar.date_input("End Date", value=pd.to_datetime("today"))
sentiment_weight = st.sidebar.slider("Sentiment Weight", 0.0, 2.0, value=1.0, step=0.1)
momentum_weight = st.sidebar.slider("Momentum Weight", 0.0, 2.0, value=1.0, step=0.1)
show_news = st.sidebar.checkbox("Show Detailed News Headlines", value=True)

# -------------------------------
# Main Analysis & Visualization
# -------------------------------
if st.sidebar.button("Analyze"):
    results = []
    combined_scores = {}
    hist_data = {}

    for company in selected_companies:
        ticker = company_name_to_ticker(company)
        if not ticker:
            results.append({
                "Company": company,
                "Ticker": "Not Found",
                "Sentiment": "N/A",
                "Momentum": "N/A",
                "Composite Score": "N/A",
                "Current Price": "N/A"
            })
        else:
            data = get_company_data(ticker)
            rt_sentiment = predict_sentiment(data["news_text"]) if data["news_text"] else {"sentiment": "No News", "confidence": 0.0}
            sentiment_score = sentiment_mapping.get(rt_sentiment["sentiment"], 0)
            hist_df = get_historical_data(ticker, start=start_date, end=end_date)
            momentum = compute_momentum(hist_df) if not hist_df.empty else 0.0
            composite_score = sentiment_weight * sentiment_score + momentum_weight * momentum
            results.append({
                "Company": company,
                "Ticker": ticker,
                "Sentiment": rt_sentiment["sentiment"],
                "Momentum": round(momentum, 4),
                "Composite Score": round(composite_score, 4),
                "Current Price": data["price_info"].get("Current Price", "N/A")
            })
            combined_scores[ticker] = composite_score
            if not hist_df.empty:
                hist_df = hist_df.rename(columns={"Close": ticker})
                hist_data[ticker] = hist_df

    # Create a results dataframe
    df_results = pd.DataFrame(results)

    # Add color coding (green for positive composite scores, red for negative)
    def color_composite(val):
        try:
            val = float(val)
            color = "green" if val > 0 else "red" if val < 0 else "black"
            return f"color: {color}"
        except:
            return ""

    st.subheader("Comparison Results")
    st.dataframe(df_results.style.applymap(color_composite, subset=["Composite Score"]))

    # Bar chart of composite scores
    if combined_scores:
        cs_df = pd.DataFrame(list(combined_scores.items()), columns=["Ticker", "Composite Score"])
        fig_cs, ax_cs = plt.subplots(figsize=(8, 4))
        bars = ax_cs.bar(cs_df["Ticker"], cs_df["Composite Score"], color=["green" if x > 0 else "red" if x < 0 else "grey" for x in cs_df["Composite Score"]])
        ax_cs.set_title("Composite Scores by Ticker")
        ax_cs.set_xlabel("Ticker")
        ax_cs.set_ylabel("Composite Score")
        st.pyplot(fig_cs)

        best_stock = max(combined_scores, key=combined_scores.get)
        st.markdown(f'<div class="recommendation"><strong>Top Recommendation:</strong> Consider reviewing <span style="font-size:20px">{best_stock}</span> (Composite Score: {round(combined_scores[best_stock],4)})</div>', unsafe_allow_html=True)

    # Create two columns for detailed analysis and plots
    col1, col2 = st.columns(2)
    with col1:
        st.subheader("Detailed Analysis per Company")
        for company in selected_companies:
            ticker = company_name_to_ticker(company)
            st.markdown(f"### {company} ({ticker if ticker else 'Ticker not found'})")
            if not ticker:
                st.warning("Ticker not found. Please update the mapping.")
            else:
                data = get_company_data(ticker)
                st.write("**Price Info:**", data["price_info"])
                if show_news:
                    st.write("**Recent News Headlines:**")
                    for i, headline in enumerate(data["news_headlines"], 1):
                        st.write(f"{i}. {headline}")
                rt_sent = predict_sentiment(data["news_text"]) if data["news_text"] else {"sentiment": "No News", "confidence": 0.0}
                st.write("**FinBERT Sentiment Analysis:**", rt_sent)
    with col2:
        st.subheader("Historical Closing Prices (3 Years)")
        if hist_data:
            for ticker, df_hist in hist_data.items():
                if not df_hist.empty:
                    plt.style.use("seaborn-darkgrid")
                    fig, ax = plt.subplots(figsize=(8, 4))
                    ax.plot(df_hist.index, df_hist[ticker], label=f"{ticker} Closing Price", color=np.random.choice(["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]))
                    ax.set_title(f"{ticker} - Historical Prices")
                    ax.set_xlabel("Date")
                    ax.set_ylabel("Closing Price")
                    ax.legend()
                    st.pyplot(fig)
        else:
            st.info("Historical data is not available for the selected companies.")


2025-04-21 18:19:25.052 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:25.060 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:25.363 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-21 18:19:25.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:25.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:25.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKE

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

2025-04-21 18:19:44.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:44.879 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:44.881 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:44.882 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:44.883 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:44.885 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:44.886 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 18:19:44.887 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [6]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.9 MB/s eta 0:00:00


In [10]:
def fetch_news_duckduckgo(query, max_results=5):
    headlines = []
    with DDGS() as ddgs:
        for r in ddgs.news(query, max_results=max_results):
            headlines.append(r['title'])
    return headlines
fetch_news_duckduckgo('apple shares')

['Apple Soars After Tariff Exemptions—And iPhones May Not Get More Expensive After All',
 'Wall Street ends higher with Apple shares as investors assess tariff exemptions',
 "Apple's Historic Selloff Has Bulls Balking From Tariff Risks",
 'Forget Tariffs, Why Apple Stock Could Be the Deal of the Decade',
 "Nvidia and Apple stocks show how investors are firefighting Trump's tariff changes"]

In [12]:
pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=d345cd924f4edef656401f8a20114b84e7d9c1651db23c089bed0a8aaf87a8e4
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [13]:
import feedparser

def get_google_news_rss(company):
    feed_url = f"https://news.google.com/rss/search?q={company}+stock"
    feed = feedparser.parse(feed_url)
    return [entry['title'] for entry in feed.entries[:10]]

In [21]:
h_AAPL=get_google_news_rss('AAPL')
d1={}
ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]
for i in range(len(ticker_list)):
    d1[ticker_list[i]]=fetch_news_duckduckgo(ticker_list[i])
print(d1)

{'AAPL': ['AAPL May Have More Problems Ahead, Expert Warns', 'SA analyst downgrades: AAPL, NVDA, MMM, CURLF, GTBIF, RJF', 'Apple (AAPL) Supplier Hails Game Changing AI Tech Putting Pressure on Rivals TSMC and Nvidia', "Apple (AAPL) to Use Samsung's OLED Screens in Its First Foldable iPhone", 'Apple: The China Fallout Has Begun'], 'MSFT': ['Microsoft Stock At $370: Opportunity Or Trap?', "Microsoft's (MSFT) AI Power Play Trumps Ongoing Tariff War", 'Microsoft Stock Is a Defensive Bet Amid Recession Fears. Why Its Price Target Was Cut.', 'Microsoft Downgraded Again: New Bearish Target Price Here', '1 "Magnificent Seven" Stock You\'ll Regret Not Buying During the Dip'], 'GOOGL': ['Google remedy hearing on search monopoly begins today. Could the tech giant get broken up?', 'Google, DOJ Go Back to Court to Fight Over Search Monopoly', 'Google faces historic antitrust trial as DOJ challenges AI search practices', 'U.S. Asks Judge to Break Up Google', "Justice Dept. asks judge to 'thaw' Googl

In [15]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [25]:
results = []
for ticker, headlines in d1.items():
    for headline in headlines:
        result = pipe(headline)  # Pass individual headline to the pipeline
        # Check if result is a list and convert to dictionary
        if isinstance(result, list):
            result = result[0]  # Assuming the first element is the relevant dictionary
        results.append({"ticker": ticker, "headline": headline, **result})  # Store ticker, headline, and result

# Aggregate sentiment
sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}
for result in results:
    label = result["label"].lower()
    sentiment_counts[label] += 1
    print(f"{result['ticker']} - {result['headline']}\n   ➤ Sentiment: {result['label']} (Confidence: {round(result['score'], 3)})\n")

# Show overall sentiment
total = sum(sentiment_counts.values())
print("=== Overall Apple Sentiment Summary ===")  # Assuming you want sentiment for AAPL
for sentiment, count in sentiment_counts.items():
    print(f"{sentiment.capitalize()}: {count} ({round(count / total * 100, 1)}%)")

overall = max(sentiment_counts, key=sentiment_counts.get)
print(f"\n📊 Overall Market Sentiment on {ticker}: **{overall.upper()}**")

AAPL - AAPL May Have More Problems Ahead, Expert Warns
   ➤ Sentiment: negative (Confidence: 0.955)

AAPL - SA analyst downgrades: AAPL, NVDA, MMM, CURLF, GTBIF, RJF
   ➤ Sentiment: negative (Confidence: 0.527)

AAPL - Apple (AAPL) Supplier Hails Game Changing AI Tech Putting Pressure on Rivals TSMC and Nvidia
   ➤ Sentiment: negative (Confidence: 0.963)

AAPL - Apple (AAPL) to Use Samsung's OLED Screens in Its First Foldable iPhone
   ➤ Sentiment: neutral (Confidence: 0.684)

AAPL - Apple: The China Fallout Has Begun
   ➤ Sentiment: negative (Confidence: 0.76)

MSFT - Microsoft Stock At $370: Opportunity Or Trap?
   ➤ Sentiment: neutral (Confidence: 0.872)

MSFT - Microsoft's (MSFT) AI Power Play Trumps Ongoing Tariff War
   ➤ Sentiment: neutral (Confidence: 0.799)

MSFT - Microsoft Stock Is a Defensive Bet Amid Recession Fears. Why Its Price Target Was Cut.
   ➤ Sentiment: negative (Confidence: 0.775)

MSFT - Microsoft Downgraded Again: New Bearish Target Price Here
   ➤ Sentiment: n

In [28]:
from transformers import pipeline
import yfinance as yf

# Load FinBERT sentiment analysis pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

def fetch_company_news(tickers):
    """
    Fetches the latest news headlines for a list of companies.

    Args:
    tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'TSLA', 'AMZN']).

    Returns:
    dict: A dictionary where the keys are ticker symbols and the values are lists of news headlines.
    """
    news_dict = {}

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        news_data = ticker.news

        if news_data:
            headlines = [article.get("title", "") for article in news_data]
            news_dict[ticker_symbol] = headlines
        else:
            news_dict[ticker_symbol] = ["No recent news found."]

    return news_dict

def analyze_sentiment_for_all_companies(tickers):
    """
    Analyzes sentiment for the news of multiple companies and returns an overall sentiment for each company.

    Args:
    tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'TSLA', 'AMZN']).

    Returns:
    dict: A dictionary where the keys are ticker symbols and the values are the overall sentiment ('positive', 'neutral', 'negative').
    """
    news_dict = fetch_company_news(tickers)

    sentiment_results = {}

    for ticker, headlines in news_dict.items():
        sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}

        # Analyze sentiment for each headline
        results = pipe(headlines)

        for result in results:
            label = result["label"].lower()
            sentiment_counts[label] += 1

        # Determine overall sentiment
        total = sum(sentiment_counts.values())
        overall_sentiment = max(sentiment_counts, key=sentiment_counts.get)

        sentiment_results[ticker] = {
            "positive": sentiment_counts["positive"],
            "neutral": sentiment_counts["neutral"],
            "negative": sentiment_counts["negative"],
            "overall_sentiment": overall_sentiment.upper()
        }

    return sentiment_results

# List of tickers to analyze
ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]

# Perform sentiment analysis for all companies in the ticker list
sentiment_analysis = analyze_sentiment_for_all_companies(ticker_list)

# Print the overall sentiment for each company
for ticker, sentiment_data in sentiment_analysis.items():
    print(f"--- Sentiment Analysis for {ticker} ---")
    print(f"Positive: {sentiment_data['positive']}")
    print(f"Neutral: {sentiment_data['neutral']}")
    print(f"Negative: {sentiment_data['negative']}")
    print(f"Overall Sentiment: {sentiment_data['overall_sentiment']}")
    print()


Device set to use cpu


--- Sentiment Analysis for AAPL ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for MSFT ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for GOOGL ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for AMZN ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for TSLA ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for META ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for NVDA ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for BRK-B ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for JPM ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for V ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--

In [30]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.1 MB/s eta 0:00:00


In [37]:
from transformers import pipeline
import yfinance as yf
import gradio as gr

# Load FinBERT pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

# Global headlines store
all_headlines = {}

# Ticker list
ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]

def fetch_company_news(tickers):
    global all_headlines
    news_dict = {}

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        news_data = ticker.news

        if news_data:
            headlines = [article.get("title", "") for article in news_data]
        else:
            headlines = ["No recent news found."]

        news_dict[ticker_symbol] = headlines

    all_headlines = news_dict
    return news_dict

def analyze_single_ticker_sentiment(ticker_symbol):
    if ticker_symbol not in all_headlines:
        fetch_company_news([ticker_symbol])

    headlines = all_headlines.get(ticker_symbol, ["No recent news found."])

    if headlines == ["No recent news found."]:
        return "No recent news available.", ""

    sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}
    results = pipe(headlines)

    output_lines = []
    for i, (headline, result) in enumerate(zip(headlines, results), 1):
        label = result["label"].lower()
        sentiment_counts[label] += 1
        output_lines.append(f"{i}. {headline}\n   ➤ Sentiment: {result['label']} (Confidence: {round(result['score'], 3)})")

    total = sum(sentiment_counts.values())
    overall = max(sentiment_counts, key=sentiment_counts.get)

    summary = f"📊 Overall Sentiment for {ticker_symbol}: {overall.upper()}\n"
    summary += f"Positive: {sentiment_counts['positive']}  |  Neutral: {sentiment_counts['neutral']}  |  Negative: {sentiment_counts['negative']}"

    return "\n\n".join(output_lines), summary

# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("# 📰 Stock News Sentiment Analysis with FinBERT")

    with gr.Row():
        ticker_input = gr.Dropdown(choices=ticker_list, label="Select a Stock Ticker")
        analyze_button = gr.Button("Analyze")

    sentiment_output = gr.Textbox(label="News Headlines & Sentiments", lines=15)
    summary_output = gr.Textbox(label="Overall Sentiment Summary")

    analyze_button.click(
        fn=analyze_single_ticker_sentiment,
        inputs=[ticker_input],
        outputs=[sentiment_output, summary_output]
    )

# Launch the app
app.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6a22342d99ef375a21.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [38]:
from transformers import pipeline
import yfinance as yf
import gradio as gr

# Load FinBERT pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

# Global headlines store
all_headlines = {}

# Ticker list
ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]

def fetch_company_news(tickers):
    global all_headlines
    news_dict = {}

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        news_data = ticker.news

        if news_data:
            headlines = [article.get("title", "") for article in news_data]
        else:
            headlines = ["No recent news found."]

        news_dict[ticker_symbol] = headlines

    all_headlines = news_dict
    return news_dict

def analyze_single_ticker_sentiment(ticker_symbol):
    if ticker_symbol not in all_headlines:
        fetch_company_news([ticker_symbol])

    headlines = all_headlines.get(ticker_symbol, ["No recent news found."])

    if headlines == ["No recent news found."]:
        return "No recent news available.", ""

    sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}
    results = pipe(headlines)

    output_lines = []
    for i, (headline, result) in enumerate(zip(headlines, results), 1):
        label = result["label"].lower()
        sentiment_counts[label] += 1
        output_lines.append(f"**{i}. {headline}**\n   ➤ Sentiment: {result['label']} (Confidence: {round(result['score'], 3)})")

    total = sum(sentiment_counts.values())
    overall = max(sentiment_counts, key=sentiment_counts.get)

    summary = f"📊 **Overall Sentiment for {ticker_symbol}: {overall.upper()}**\n"
    summary += f"Positive: {sentiment_counts['positive']}  |  Neutral: {sentiment_counts['neutral']}  |  Negative: {sentiment_counts['negative']}"

    return "\n\n".join(output_lines), summary

# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("# 📰 Stock News Sentiment Analysis with FinBERT")

    with gr.Row():
        ticker_input = gr.Dropdown(choices=ticker_list, label="Select a Stock Ticker")
        analyze_button = gr.Button("Analyze")

    sentiment_output = gr.Textbox(label="News Headlines & Sentiments", lines=15)
    summary_output = gr.Textbox(label="Overall Sentiment Summary")

    analyze_button.click(
        fn=analyze_single_ticker_sentiment,
        inputs=[ticker_input],
        outputs=[sentiment_output, summary_output]
    )

# Launch the app
app.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f4393fc0c37a1dcca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [44]:
import yfinance as yf

# List of tickers
ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]

def fetch_profit_loss(tickers, period='1y'):
    """
    Fetches profit or loss percentage for a list of stock tickers over a given period.

    Args:
    tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'MSFT', 'AMZN']).
    period (str): Time period for calculating profit/loss ('1d', '5d', '1mo', '3mo', '1y').

    Returns:
    dict: A dictionary with tickers as keys and profit/loss percentages as values.
    """
    profit_loss_dict = {}

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)

        # Get historical market data for the desired period
        hist_data = ticker.history(period=period)

        # Get the opening price from the start of the period and the current closing price
        start_price = hist_data['Close'].iloc[0]
        end_price = hist_data['Close'].iloc[-1]

        # Calculate the percentage change (Profit or Loss)
        profit_loss_percentage = ((end_price - start_price) / start_price) * 100
        profit_loss_dict[ticker_symbol] = profit_loss_percentage

    return profit_loss_dict

# Fetch profit/loss for all companies in the ticker list over 1 year
profit_loss_percentage = fetch_profit_loss(ticker_list, period='3mo')

# Print the profit/loss percentage for each company
for ticker, pl_percentage in profit_loss_percentage.items():
    print(f"{ticker}: {pl_percentage:.2f}%")


AAPL: -13.61%
MSFT: -19.35%
GOOGL: -25.47%
AMZN: -28.80%
TSLA: -45.20%
META: -22.20%
NVDA: -34.10%
BRK-B: 10.08%
JPM: -12.34%
V: -0.90%
PG: 1.23%
JNJ: 8.88%
UNH: -17.81%
HD: -14.87%
MA: -3.23%
DIS: -22.80%
PYPL: -32.90%
NFLX: 3.56%
INTC: -13.82%
PEP: -3.45%


In [49]:
import gradio as gr
from transformers import pipeline
import yfinance as yf

# Load FinBERT sentiment pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

# Mapping of full company names to tickers
company_dict = {
    "Apple Inc.": "AAPL",
    "Microsoft Corporation": "MSFT",
    "Alphabet Inc. (Google)": "GOOGL",
    "Amazon.com, Inc.": "AMZN",
    "Tesla, Inc.": "TSLA",
    "Meta Platforms, Inc.": "META",
    "NVIDIA Corporation": "NVDA",
    "Berkshire Hathaway Inc.": "BRK-B",
    "JPMorgan Chase & Co.": "JPM",
    "Visa Inc.": "V",
    "Procter & Gamble Co.": "PG",
    "Johnson & Johnson": "JNJ",
    "UnitedHealth Group Incorporated": "UNH",
    "The Home Depot, Inc.": "HD",
    "Mastercard Incorporated": "MA",
    "The Walt Disney Company": "DIS",
    "PayPal Holdings, Inc.": "PYPL",
    "Netflix, Inc.": "NFLX",
    "Intel Corporation": "INTC",
    "PepsiCo, Inc.": "PEP"
}

# Fetch news headlines for a ticker
def fetch_news(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    news_data = ticker.news
    return [article.get("title", "") for article in news_data] if news_data else []

# Analyze sentiment of headlines
def analyze_sentiment(headlines):
    sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}
    if headlines:
        results = pipe(headlines)
        for result in results:
            label = result["label"].lower()
            sentiment_counts[label] += 1
        overall_sentiment = max(sentiment_counts, key=sentiment_counts.get).upper()
    else:
        overall_sentiment = "NO DATA"
    return sentiment_counts, overall_sentiment

# Calculate profit/loss
def fetch_profit_loss(ticker, period='3mo'):
    hist_data = yf.Ticker(ticker).history(period=period)
    if hist_data.empty:
        return None
    start_price = hist_data['Close'].iloc[0]
    end_price = hist_data['Close'].iloc[-1]
    return ((end_price - start_price) / start_price) * 100

# Combined Gradio function
def analyze_company(company_name):
    ticker = company_dict[company_name]
    headlines = fetch_news(ticker)
    sentiment_counts, overall_sentiment = analyze_sentiment(headlines)
    profit_loss = fetch_profit_loss(ticker)

    sentiment_output = (
        f"📰 **Sentiment Analysis for {company_name} ({ticker})**\n"
        f"Positive: {sentiment_counts['positive']}\n"
        f"Neutral: {sentiment_counts['neutral']}\n"
        f"Negative: {sentiment_counts['negative']}\n"
        f"Overall: {overall_sentiment}\n"
    )

    pl_output = (
        f"\n💹 **Profit/Loss for last 3 months**: "
        f"{profit_loss:.2f}%" if profit_loss is not None else "\n💹 No price data available."
    )

    news_output = "\n\n🗞 **Top News Headlines:**\n" + "\n".join([f"- {h}" for h in headlines[:5]]) if headlines else "\n\n🗞 No recent headlines found."

    return sentiment_output + pl_output + news_output

# Gradio app
with gr.Blocks() as demo:
    gr.Markdown("## 📊 Company Market Sentiment & Performance Tracker")
    company_dropdown = gr.Dropdown(
        choices=list(company_dict.keys()),
        label="Select a Company",
        interactive=True
    )
    output_box = gr.Textbox(label="Analysis Output", lines=20, interactive=False)
    company_dropdown.change(fn=analyze_company, inputs=company_dropdown, outputs=output_box)

demo.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3f2ebadfbaf4d9759e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
